In [59]:
import folium
import pandas as pd
import numpy as np
import networkx as nx

# data load
nodes = pd.read_csv('./data/bike_node(21.01.31).csv').drop('Unnamed: 0', 1)
links = pd.read_csv('./data/group_on.csv').drop('Unnamed: 0', 1)

# x 보다 큰 것만 생각하기
def new_links(x, n):
    x = x[x['sum'] > n]
    return x

In [63]:
links1 = new_links(links, 10)

In [64]:
# folium node plot
map1 = folium.Map(location = (37.5642135, 127.0016985), zoom_start = 11, tiles = 'cartodbpositron')

for ix, row in nodes.iterrows():
    location = (row['latitude'], row['longitude'])
    folium.Circle(
        location = location,
        radius = 2,
        color = 'red',
        weight = 1,
        fill_color = 'red',
        fill = True
    ).add_to(map1)

for ix, row in links1.iterrows():
    start = tuple(nodes[nodes['id'] == row['start']][['latitude', 'longitude']].iloc[0])
    end = tuple(nodes[nodes['id'] == row['end']][['latitude', 'longitude']].iloc[0])
    folium.PolyLine(
        locations = [start, end],
        color = 'green',
        weight = 1,
        line_cap = 'round'
    ).add_to(map1)

map1